In [1]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment, EnvironmentSettings
import os

env = StreamExecutionEnvironment.get_execution_environment()
# Kafka 파티션 수
env.set_parallelism(3)

settings = EnvironmentSettings.in_streaming_mode()
t_env = StreamTableEnvironment.create(env, environment_settings=settings)
t_env.get_config().set_local_timezone("Asia/Seoul")

# set the checkpoint mode to EXACTLY_ONCE
t_env.get_config().get_configuration().set_string("execution.checkpointing.mode", "EXACTLY_ONCE")
t_env.get_config().get_configuration().set_string("execution.checkpointing.interval", "1 min")

# set the statebackend type to "rocksdb", other available options are "hashmap"
#t_env.get_config().set("state.backend.type", "rocksdb")

# set the checkpoint directory, which is required by the RocksDB statebackend
# 작업 종료 시에도 CheckPoint를 삭제되지 않기 위해 "RETAIN_ON_CANCELLATION" 사용
#t_env.get_config().set("execution.checkpointing.dir", "s3a://pyflink-test-hs/checkpoint/")
#t_env.get_config().set("execution.checkpointing.externalized-checkpoint-retention", "RETAIN_ON_CANCELLATION")

In [ ]:
# 2. Kafka 소스 테이블 생성
t_env.execute_sql("""
CREATE TABLE upbit_transaction (
    type STRING,
    code STRING,
    opening_price DOUBLE,
    high_price DOUBLE,
    low_price DOUBLE,
    trade_price DOUBLE,
    prev_closing_price DOUBLE,
    acc_trade_price DOUBLE,
    change STRING,
    change_price DOUBLE,
    signed_change_price DOUBLE,
    change_rate DOUBLE,
    signed_change_rate DOUBLE,
    ask_bid STRING,
    trade_volume DOUBLE,
    acc_trade_volume DOUBLE,
    trade_date STRING,
    trade_time STRING,
    trade_timestamp BIGINT,
    acc_ask_volume DOUBLE,
    acc_bid_volume DOUBLE,
    highest_52_week_price DOUBLE,
    highest_52_week_date STRING,
    lowest_52_week_price DOUBLE,
    lowest_52_week_date STRING,
    market_state STRING,
    is_trading_suspended BOOLEAN,
    delisting_date STRING,
    market_warning STRING,
    `timestamp` BIGINT,
    acc_trade_price_24h DOUBLE,
    acc_trade_volume_24h DOUBLE,
    stream_type STRING,
    name STRING,
    event_time AS TO_TIMESTAMP_LTZ(`timestamp`, 3),
    WATERMARK FOR event_time AS event_time - INTERVAL '1' SECOND
) WITH (
    'connector' = 'kafka',
    'topic' = 'upbit',
    'properties.bootstrap.servers' = '13.124.100.215:9091',
    'properties.group.id' = 'flink-upbit-group',
    'scan.startup.mode' = 'latest-offset',
    'format' = 'json',
    'json.fail-on-missing-field' = 'false',
    'json.ignore-parse-errors' = 'true'
)
""")

table_result1 = t_env.execute_sql("SELECT * FROM upbit_transaction")
table_result1.print()

+----+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+----------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+----------------------+--------------------------------+--------------------------------+----------------------+--------------------------------+-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/mzc01-hyucksangcho/opt/anaconda3/envs/pyflink_38/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/mzc01-hyucksangcho/opt/anaconda3/envs/pyflink_38/lib/python3.8/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/mzc01-hyucksangcho/opt/anaconda3/envs/pyflink_38/lib/python3.8/socket.py", line 681, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

| +I |                         ticker |                       KRW-OMNI |                         5590.0 |                         5605.0 |                         5265.0 |                         5440.0 |                         5600.0 |            9.351399766360815E9 |                           FALL |                          160.0 |                         -160.0 |                   0.0285714286 |                  -0.0285714286 |                            BID |                    43.95540939 |               1736977.22296382 |                       20250806 |                         140154 |        1754488914956 |               1036440.96167317 |                700536.26129065 |                        11400.0 |                     2025-07-29 |                         5265.0 |                     2025-08-06 |                         ACTIVE |                FALSE |                         <NULL> |                           NONE |        1754488914984 |           1.335996369137431E10 | 

In [6]:
import pyflink
import os
print(os.path.dirname(os.path.abspath(pyflink.__file__)))


/Users/mzc01-hyucksangcho/opt/anaconda3/envs/pyflink_38/lib/python3.8/site-packages/pyflink
